# Advanced RAG Pipeline - Different Retriever Implementations

This notebook demonstrates various retrieval strategies using the VectorStoreService class:
1. **Naive/Basic Retriever** - Simple similarity search
2. **Contextual Compression with Reranking** - Using Cohere reranker to improve results
3. **Multi-Query Retriever** - Generating multiple queries for better coverage
4. **Ensemble Retriever** - Combining multiple retrieval methods
5. **Parent Document Retriever** - Retrieving larger context chunks
6. **Self-Query Retriever** - Query with metadata filtering

We'll use PDF documents from the resources directory for testing.

## Setup and Imports

In [ ]:
import os
import sys
from pathlib import Path
import asyncio
from typing import List, Dict

# Add parent directory to path
sys.path.append(str(Path.cwd().parent))

# LangChain imports
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Qdrant
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.retrievers import (
    ContextualCompressionRetriever,
    MultiQueryRetriever,
    EnsembleRetriever,
    ParentDocumentRetriever
)
from langchain.retrievers.document_compressors import (
    LLMChainExtractor,
    EmbeddingsFilter
)
from langchain.storage import InMemoryStore
from langchain_community.retrievers import BM25Retriever
from qdrant_client import QdrantClient

# Try to import Cohere for reranking (optional)
try:
    from langchain.retrievers.document_compressors import CohereRerank
    COHERE_AVAILABLE = True
except ImportError:
    COHERE_AVAILABLE = False
    print("Cohere not available. Install with: pip install cohere")

# Import our custom service
from app.services.vector_store import VectorStoreService
from app.config import settings

## Load and Process Documents

In [ ]:
# Example: Hybrid retriever with filtering and compression
# Step 1: Use ensemble for diverse results
# Step 2: Filter by threshold
# Step 3: Compress with LLM

hybrid_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=ContextualCompressionRetriever(
        base_compressor=embeddings_filter,
        base_retriever=ensemble_retriever
    )
)

hybrid_results = hybrid_retriever.invoke(TEST_QUERY)

print("=" * 80)
print("HYBRID RETRIEVER (Ensemble + Filter + Compression)")
print("=" * 80)
print(f"Retrieved {len(hybrid_results)} documents\n")
for i, doc in enumerate(hybrid_results, 1):
    print(f"\n[{i}] Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Content: {doc.page_content}")
    print("-" * 80)

## Extending Retrievers - Custom Example

You can combine multiple techniques to create custom retrievers tailored to your needs.

## Summary

### Retriever Comparison:

1. **Basic Retriever**: Fast and simple, good baseline
2. **Contextual Compression (LLM)**: More precise but slower, extracts only relevant parts
3. **Embeddings Filter**: Fast filtering based on similarity threshold
4. **Cohere Rerank**: Best quality but requires API, excellent for reordering results
5. **Multi-Query**: Better coverage through query expansion
6. **Ensemble**: Combines semantic and keyword search for robustness
7. **Parent Document**: Good for returning larger context
8. **MMR**: Best for diverse results, avoids redundancy

### Recommendations:
- **For speed**: Basic or Embeddings Filter
- **For quality**: Cohere Rerank or Contextual Compression
- **For coverage**: Multi-Query or Ensemble
- **For diversity**: MMR
- **For context**: Parent Document Retriever

### Key Takeaways:
- Different retrievers excel at different tasks
- Ensemble methods often provide the best balance
- Consider your use case: speed vs quality vs coverage
- The VectorStoreService class provides a simple interface for basic retrieval
- Advanced retrievers can significantly improve RAG performance

In [ ]:
# Compare RAG chains
rag_chains = {
    "Basic RAG": create_rag_chain(basic_retriever),
    "Compressed RAG": create_rag_chain(compression_retriever),
    "Ensemble RAG": create_rag_chain(ensemble_retriever),
    "Multi-Query RAG": create_rag_chain(multiquery_retriever),
}

print("=" * 80)
print("RAG CHAIN COMPARISON")
print("=" * 80)
print(f"Question: {test_question}\n")

for name, chain in rag_chains.items():
    print("\n" + "=" * 80)
    print(name)
    print("=" * 80)
    
    try:
        result = chain.invoke({"query": test_question})
        print(f"\nAnswer: {result['result']}")
        print(f"\nSources: {len(result['source_documents'])} documents")
        for i, doc in enumerate(result['source_documents'][:2], 1):
            print(f"  [{i}] {doc.metadata.get('source', 'Unknown')}")
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# RAG prompt
template = """Use the following context to answer the question.
If you don't know, say so.

Context:
{context}

Question: {question}

Answer:"""

QA_PROMPT = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

def create_rag_chain(retriever):
    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": QA_PROMPT}
    )

test_question = "What are the most important principles for writing clear mathematical papers?"

## Complete RAG Chain

Building end-to-end RAG pipelines with different retrievers.

In [ ]:
# Similarity search
query = "What are best practices for mathematical writing?"
results = await vector_service.similarity_search(query, k=5)

print("=" * 80)
print("VectorStoreService RESULTS")
print("=" * 80)
print(f"Query: {query}\n")

for i, result in enumerate(results, 1):
    print(f"\n[{i}] Score: {result['score']:.4f}")
    print(f"Source: {result['metadata'].get('source', 'Unknown')}")
    print(f"Content: {result['text'][:300]}...")
    print("-" * 80)

In [ ]:
# Initialize service
vector_service = VectorStoreService()

# Process PDFs
print("Processing PDFs with VectorStoreService...")
total_chunks = 0

for pdf_path in pdf_files:
    metadata = {"source": pdf_path.name}
    chunk_count = await vector_service.process_pdf(str(pdf_path), metadata)
    total_chunks += chunk_count
    print(f"  {pdf_path.name}: {chunk_count} chunks")

print(f"\nTotal: {total_chunks} chunks indexed")

## Using VectorStoreService Class

Demonstrates how to use the custom VectorStoreService class from the repository.

In [ ]:
test_queries = [
    "What are the key principles of mathematical writing?",
    "How should one structure a mathematical proof?",
    "What is the importance of clarity in technical writing?",
]

retrievers_to_compare = {
    "Basic": basic_retriever,
    "Compressed": compression_retriever,
    "Filter": filter_retriever,
    "Multi-Query": multiquery_retriever,
    "Ensemble": ensemble_retriever,
    "MMR": mmr_retriever,
}

print("=" * 80)
print("RETRIEVER COMPARISON")
print("=" * 80)

for query in test_queries:
    print(f"\n\nQuery: {query}")
    print("-" * 80)
    
    for name, retriever in retrievers_to_compare.items():
        try:
            results = retriever.invoke(query)
            print(f"{name:15} -> {len(results)} documents")
        except Exception as e:
            print(f"{name:15} -> Error: {str(e)[:40]}")

## Comparison Across Multiple Queries

In [ ]:
mmr_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 5,
        "fetch_k": 20,
        "lambda_mult": 0.5
    }
)

mmr_results = mmr_retriever.invoke(TEST_QUERY)

print("=" * 80)
print("MMR RETRIEVER RESULTS")
print("=" * 80)
for i, doc in enumerate(mmr_results, 1):
    print(f"\n[{i}] Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Content: {doc.page_content[:300]}...")
    print("-" * 80)

## 8. MMR (Maximum Marginal Relevance)

Balances relevance and diversity in results.

In [ ]:
parent_results = parent_retriever.invoke(TEST_QUERY)

print("=" * 80)
print("PARENT DOCUMENT RETRIEVER RESULTS")
print("=" * 80)
print(f"Retrieved {len(parent_results)} parent documents\n")
for i, doc in enumerate(parent_results, 1):
    print(f"\n[{i}] Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Length: {len(doc.page_content)} chars")
    print(f"Preview: {doc.page_content[:300]}...")
    print("-" * 80)

In [ ]:
# Parent and child splitters
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

# Store for parents
store = InMemoryStore()

# Child vector store
child_vectorstore = Qdrant.from_documents(
    [],
    embeddings,
    collection_name="child_chunks",
    client=qdrant_client,
    force_recreate=True
)

# Parent retriever
parent_retriever = ParentDocumentRetriever(
    vectorstore=child_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

parent_retriever.add_documents(all_documents)
print(f"Store contains {len(store.store)} parent documents")

## 7. Parent Document Retriever

Retrieves small chunks for similarity but returns larger parent documents for context.

In [ ]:
# BM25 retriever (keyword-based)
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 5

# Ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[basic_retriever, bm25_retriever],
    weights=[0.5, 0.5]
)

ensemble_results = ensemble_retriever.invoke(TEST_QUERY)

print("=" * 80)
print("ENSEMBLE RETRIEVER RESULTS")
print("=" * 80)
for i, doc in enumerate(ensemble_results, 1):
    print(f"\n[{i}] Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Content: {doc.page_content[:300]}...")
    print("-" * 80)

## 6. Ensemble Retriever

Combines vector search + BM25 keyword search using Reciprocal Rank Fusion.

In [ ]:
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    llm=llm
)

multiquery_results = multiquery_retriever.invoke(TEST_QUERY)

print("\n" + "=" * 80)
print("MULTI-QUERY RETRIEVER RESULTS")
print("=" * 80)
print(f"Retrieved {len(multiquery_results)} unique documents\n")
for i, doc in enumerate(multiquery_results, 1):
    print(f"\n[{i}] Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Content: {doc.page_content[:300]}...")
    print("-" * 80)

## 5. Multi-Query Retriever

Generates multiple query variations for better coverage.

In [ ]:
if COHERE_AVAILABLE and os.getenv("COHERE_API_KEY"):
    cohere_reranker = CohereRerank(
        model="rerank-english-v2.0",
        top_n=5
    )
    
    rerank_retriever = ContextualCompressionRetriever(
        base_compressor=cohere_reranker,
        base_retriever=vectorstore.as_retriever(search_kwargs={"k": 20})
    )
    
    reranked_results = rerank_retriever.invoke(TEST_QUERY)
    
    print("=" * 80)
    print("COHERE RERANK RESULTS")
    print("=" * 80)
    for i, doc in enumerate(reranked_results, 1):
        print(f"\n[{i}] Source: {doc.metadata.get('source', 'Unknown')}")
        print(f"Score: {doc.metadata.get('relevance_score', 'N/A')}")
        print(f"Content: {doc.page_content[:300]}...")
        print("-" * 80)
else:
    print("Cohere not available or no API key set")

## 4. Cohere Rerank (Optional)

Uses Cohere's reranking model if available.

In [ ]:
# Embeddings filter
embeddings_filter = EmbeddingsFilter(
    embeddings=embeddings,
    similarity_threshold=0.76
)

filter_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter,
    base_retriever=vectorstore.as_retriever(search_kwargs={"k": 10})
)

filtered_results = filter_retriever.invoke(TEST_QUERY)

print("=" * 80)
print("EMBEDDINGS FILTER RESULTS")
print("=" * 80)
print(f"Retrieved {len(filtered_results)} documents after filtering\n")
for i, doc in enumerate(filtered_results, 1):
    print(f"\n[{i}] Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Content: {doc.page_content[:300]}...")
    print("-" * 80)

## 3. Embeddings Filter

Filters based on similarity threshold.

In [ ]:
# LLM chain extractor
compressor = LLMChainExtractor.from_llm(llm)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=basic_retriever
)

compressed_results = compression_retriever.invoke(TEST_QUERY)

print("=" * 80)
print("CONTEXTUAL COMPRESSION RESULTS")
print("=" * 80)
for i, doc in enumerate(compressed_results, 1):
    print(f"\n[{i}] Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Content: {doc.page_content}")
    print("-" * 80)

## 2. Contextual Compression with LLM

Uses LLM to extract only relevant parts.

In [ ]:
# Basic retriever
basic_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

# Retrieve documents
basic_results = basic_retriever.invoke(TEST_QUERY)

print("=" * 80)
print("BASIC RETRIEVER RESULTS")
print("=" * 80)
for i, doc in enumerate(basic_results, 1):
    print(f"\n[{i}] Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Content: {doc.page_content[:300]}...")
    print("-" * 80)

## 1. Naive/Basic Retriever

Simple similarity search on embeddings.

In [ ]:
# Initialize LLM
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# Test query
TEST_QUERY = "What are the key principles of mathematical writing?"
print(f"Test Query: {TEST_QUERY}")

In [ ]:
# Initialize embeddings
embeddings = OpenAIEmbeddings(
    model=settings.embedding_model,
    openai_api_key=settings.openai_api_key
)

# Initialize Qdrant
qdrant_client = QdrantClient(
    host=settings.qdrant_host,
    port=settings.qdrant_port
)

# Create vector store
collection_name = "rag_test_collection"

vectorstore = Qdrant.from_documents(
    chunks,
    embeddings,
    collection_name=collection_name,
    client=qdrant_client,
    force_recreate=True
)

print(f"Vector store created with {len(chunks)} documents")

## Initialize Vector Store

In [ ]:
# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

chunks = text_splitter.split_documents(all_documents)
print(f"Created {len(chunks)} chunks")
print(f"\nExample chunk:")
print(f"Content: {chunks[0].page_content[:200]}...")
print(f"Metadata: {chunks[0].metadata}")

In [ ]:
# Load all PDFs
all_documents = []

for pdf_path in pdf_files:
    loader = PyPDFLoader(str(pdf_path))
    docs = loader.load()
    for doc in docs:
        doc.metadata["source"] = pdf_path.name
    all_documents.extend(docs)

print(f"Loaded {len(all_documents)} pages total")

In [ ]:
# Define resources directory
RESOURCES_DIR = Path("resources")

# List available PDFs
pdf_files = list(RESOURCES_DIR.glob("*.pdf"))
print(f"Found {len(pdf_files)} PDF files:")
for pdf in pdf_files:
    print(f"  - {pdf.name}")